# Part 0. Zero to Model in 14 Lines of code

Prerequisites: In a terminal please run:

conda env create -f conda_env.yml 

conda activate snowpark-ml-hol

### 0.0 Import our libraries

In [5]:
import json
import pandas as pd
from snowflake.snowpark.session import Session
import snowflake.snowpark.functions as F
from snowflake.ml.modeling.linear_model import LinearRegression

### 0.1 Reading Snowflake Connection Details, create a Session


In [2]:
snowflake_connection_cfg = json.loads(open("/Users/mitaylor/Documents/creds/creds_sf_azure_trial.json").read())
session = Session.builder.configs(snowflake_connection_cfg).create()

### 0.2 Specify Your Database and Create a Virtual Warehouse

In [3]:
session.sql("CREATE OR REPLACE DATABASE HOL_DEMO").collect()
session.sql("CREATE OR REPLACE WAREHOUSE ASYNC_WH WITH WAREHOUSE_SIZE='MEDIUM' WAREHOUSE_TYPE = 'SNOWPARK-OPTIMIZED'").collect()

[Row(status='Warehouse ASYNC_WH successfully created.')]

### 0.3 Load Some Data

In [6]:
session.write_pandas(pd.read_csv("test.csv"), table_name='FS_DATASET', auto_create_table=True, overwrite=True)
sdf = session.table("FS_DATASET")

### 0.4 Train Your Model

In [6]:
regressor = LinearRegression(input_cols=["OPEN"], label_cols=["CLOSE"],output_cols=["CLOSE_PREDICT"])
regressor.fit(sdf)

### 0.5 Run Your Model

In [8]:
regressor.predict(sdf).limit(3).to_pandas()

,LOW,DATE,HIGH,SYMBOL,OPEN,CLOSE,CLOSE_PREDICT
0,73.797501,2020-01-02,75.150002,IBM,74.059998,75.087502,74.325285
1,74.125000,2020-01-03,75.144997,IBM,74.287498,74.357498,74.552333
2,73.187500,2020-01-06,74.989998,IBM,73.447502,74.949997,73.714009


# Conclusion

Obviously, we're missing a bunch of things that would normally take place like train-test splits, saving results, cross validation, hyperparameter search etc., but this should get most people the headstart they need to start playing with the wider Snowpark ecosystem.  On to Lab 1.!